In [2]:
import pyfastx
import pandas as pd
import re
import altair as alt
import numpy as np
from altair_saver import save
alt.data_transformers.disable_max_rows()
import regex

# Average polyA and polyT length

In [77]:
fq = pyfastx.Fastq("/home/nanopore/sequencing-data/PolyA_2022-11-23/merged_fastq/PolyA_2022-11-23_merged.fastq.gz",
                  build_index=False)

In [78]:
poly_t = re.compile(r"(TTTTTTTTTTTTT*.?T*).")
poly_a = re.compile(r"(AAAAAAAAAAAAA*.?T*).")

In [90]:
t_lengths = []
t_ids = []
for read in fq:
    if "TTTTTTTTTTTTT" in read[1][:100]:
        match = re.findall(poly_t, read[1])
        t_lengths.append(len(match[0]))
        t_ids.append(read[0])

In [91]:
a_lengths = []
a_ids = []
for read in fq:
    if "AAAAAAAAAAAAA" in read[1][-100:]:
        match = re.findall(poly_a, read[1])
        a_lengths.append(len(match[0]))
        a_ids.append(read[0])

In [98]:
df_t = pd.DataFrame({"ts": t_lengths, "id": t_ids})
df_a = pd.DataFrame({"as": a_lengths, "id": a_ids})

In [108]:
both = pd.concat([df_t, df_a])
both.shape[0] - both.drop_duplicates("id").shape[0]
#both.drop_duplicates("id")

31215

In [99]:
pd.concat([df_t["ts"].to_frame().describe().T, df_a["as"].to_frame().describe().T])

,count,mean,std,min,25%,50%,75%,max
ts,252129.0,32.320415,8.492304,13.0,28.0,32.0,38.0,155.0
as,153487.0,25.044623,13.290954,12.0,18.0,23.0,29.0,445.0


In [50]:
poly_t = alt.Chart(df_t).mark_area().encode(
 alt.X("ts:Q", bin=alt.Bin(extent=[15, 300], step=10), title="size (bp)"),
 alt.Y("count()", title="number of reads with this length")
).properties(width=600,
             height=400,
             title="Histogram of polyT-tail")

In [49]:
poly_a = alt.Chart(df_a).mark_area(color="orange").encode(
 alt.X("as:Q", bin=alt.Bin(extent=[15, 300], step=10), title="size (bp)"),
 alt.Y("count()", title="number of reads with this length"),
).properties(width=600,
             height=400,
             title="Histogram of polyA-tail")

In [52]:
both = poly_a & poly_t

In [84]:
title = (
    (f"Reads with PolyA and Poly T tails."),
    (f"polyA: Number: {n_poly_a}. Mean: {mean_poly_a}"),
    (f"polyT: Number: {n_poly_t}. Mean: {mean_poly_t}")
)

In [87]:
both = both.properties(title=title)

In [89]:
save(both, "poly-tail-lengths.pdf")

In [69]:
test = "AAATTACGTATTGCTACAATGGGAAGAGCACACGTCTTTTTTTTTTTTTTTTCCCTAAAAAAAAAAAATTAAAAAAAAAAAAAAAATAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAGAGCGTGTGTCTCCCGATCTAGCAAGAGAT"
base = r".{0,2}TTTTT*" * 2

pattern = re.compile(f"({base}).")
match = re.findall(pattern, test)
match



['TCTTTTTTTTTTTTTTTT']

# Trying the `regex` library for fuzzy matching 

In [95]:
text = "AAAATTAAAAGGG"
pattern = r"(AAA*.?A+)."
fuzzy_pattern = f'{pattern}{{e<=7}}'

match = regex.findall(fuzzy_pattern, text)
match

['AAAA', 'AAAA']

# Trying the `re` module again, but allow 1 mistake in the middle 

In [83]:
poly_t_pattern = re.compile(r"(TTTTTT*.?TTTT*.?TTTTT*.?TTTTT*).")
poly_a_pattern = re.compile(r"(AAAAAA+.?AAAA+).")

In [85]:
%%time
t_lengths = []
for read in fq:
    if "TTTTTTTTTTTTT" in read[1][:100] and len(read[1]) > 200:
        match = re.findall(poly_t_pattern, read[1])
        t_lengths.append(len(match[0]))

IndexError: list index out of range

# How many reads have both polyA and polyT and are over 500 bp?

In [94]:
total = 0
target = 0
for read in fq:
    total += 1
    if "TTTTTTTTTTTTT" in read[1][:100] and "AAAAAAAAAAAAA" in read[1][-100:] and len(read[1]) > 500:
        target += 1

In [96]:
percent = (target / total) * 100
percent

0.7319585106564683

# Wrangle the `telomere-regions` file to the format Greider uses

### chrx == chr23 and chry == chr24

In [3]:
import pandas as pd
import numpy as np

In [21]:
df = pd.read_csv("telomere-regions-chm13v2.csv")

In [22]:
(df
 .assign(chr=lambda x: np.select([x.chr == "chrX", x.chr == "chrY"],
                                  ["chr23", "chr24"],
                                  default=x.chr),
         )
 .assign(chr=lambda x: np.select([x.end == "left", 
                                  x.end == "right"],
                                 [x.chr.str.replace("chr", "") + "L",
                                  x.chr.str.replace("chr", "") + "R"],
                                 default=pd.NA))
 .loc[:, ["chr", "chromEnd"]]
 .rename(columns={"chr": "chromosome_arm",
          "chromEnd": "telomere_position"})
 .to_csv("chromosome-starts-chm13v2.csv", index=False)
)

# Looking at a read with telomere

In [38]:
for read in fq: 
    if read[0] == "83c29e37-0983-4e27-aa50-0a9e17cc00fe":
        sequence = read[1]
    

In [25]:
# It is around 9kb long!
sequence[0:1000]

NameError: name 'sequence' is not defined

In [129]:
sequence[:100]

t_pattern = re.compile(r"TTTTTT*")

re.

'GTTATGTGGTTTACTTGGTTGGTTACATTGTACAATGGAAGAGCACACATCTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTT'

In [111]:
sequence[18000:19000].count("C")

222

In [117]:
reversed(sequence)

In [127]:
test = "ABCDEFGHIJKLMNOPQRSTUVV"

test = reversed(test)

for chunk in more_itertools.windowed(test, 3, step=3):
    print(chunk.count("V"))


2
0
0
0
0
0
0
0


In [26]:
test = "hejsanhejsan"
pattern = re.compile(r"ejsa")

re.search(pattern, test).span()[1]
test[::-1][1:]

np.

'asjehnasjeh'

### function for identifying length of telomere

In [54]:
import numpy as np
import re
import more_itertools

def extract_telomere_length(sequence: str, end: str, threshold: float = 0.3, 
                            window: int = 50
) -> int:
    """
    Extract telomere length of read by looking at G and C content.
    If G | C content > treshold, then telomere
    
    :param sequence str: Sequence of interest
    :param threshold float: Threshold of G | C content. Default 0.30
    :param window int: Size of sliding window. Default 50
    :param end str: End of the chromosme. L or R
    :return int: Length of telomere
    """
    if end not in ["L", "R"]:
        print("Please provide L or R as argument for end")
        #exit(1)
    
    nucleotide = "C" if end == "L" else "G"
    sequence = sequence if end == "L" else sequence[::-1]
    # find the where the telomere starts (after the polyA tail)
    pattern = re.compile(r"TTTTTTTTTT*") if end == "L" else re.compile(r"AAAAAAAAAA*")
    telomere_start = re.search(pattern, sequence).span()[1]
    sequence = sequence[telomere_start:]
    
    telomere_length = 0
    for chunk in more_itertools.windowed(sequence, window, step=window):
        if chunk.count(nucleotide) / window > threshold:
            telomere_length += window
        else:
            break
        
    return telomere_length

extract_telomere_length(sequence, end="L")

8100

## Look at the Tidehunter output

In [34]:
import pandas as pd

tidehunter = (
    pd.read_csv("/home/nanopore/analyzed-data/PolyA_2022-11-23/tidehunter/PolyA_2022-11-23_raw_tidehunter.out",
    sep="\t",
    header=None,
    names=["id", "rep_number", "copy_number", "read_len", 
           "start", "end", "conslen", "avematch", "full_len", "subpos", "cons_seq"]
    )
   .assign(region_length=lambda x: x.end - x.start) 
   .drop(columns=["rep_number", "copy_number", "full_len", "subpos"])
)

In [45]:
# PolyA tail
# How can certain reads START from the beginning?
(tidehunter
 .loc[lambda x: ~x.cons_seq.str.contains("T|G|C")]
 #.loc[lambda x: x.start < 200]
 #.loc[lambda x: x.copy_number > 10]
 .sort_values("region_length", ascending=False)
 .drop_duplicates("id")
)


,id,read_len,start,end,conslen,avematch,cons_seq,region_length
518979,7d2975a5-e8cb-4e7b-a203-d740f6060a58,5552,4650,5312,30,100.0,AAAAAAAAAAAAAAAAAAAAAAAAAAAAAA,662
47263,261e6a43-129f-4ebe-baa7-ee380c6f7dfe,26413,25943,26390,30,100.0,AAAAAAAAAAAAAAAAAAAAAAAAAAAAAA,447
330952,1d3a52d6-236b-42a3-bc27-76260efa358a,16582,15559,15995,30,100.0,AAAAAAAAAAAAAAAAAAAAAAAAAAAAAA,436
175996,e47dc51b-14bf-47a1-9f9b-aaed545bf2dd,7155,5873,6299,30,100.0,AAAAAAAAAAAAAAAAAAAAAAAAAAAAAA,426
5987,50e8f5e2-5364-4b88-aa6e-eaa6a80304de,2582,1488,1909,30,100.0,AAAAAAAAAAAAAAAAAAAAAAAAAAAAAA,421
...,...,...,...,...,...,...,...,...
7930,b4211005-6dc6-4deb-a7ea-26f37b6d8e43,29823,28826,28893,30,100.0,AAAAAAAAAAAAAAAAAAAAAAAAAAAAAA,67
147025,8f4062ac-8532-4633-8c85-d9736602fec0,1141,1051,1118,30,100.0,AAAAAAAAAAAAAAAAAAAAAAAAAAAAAA,67
415983,9fbd8e06-e798-4c4e-bbf3-5434bea4aead,15649,11,78,30,100.0,AAAAAAAAAAAAAAAAAAAAAAAAAAAAAA,67
411853,334764b6-4b1e-4237-b115-7e9c62596155,18003,17936,18003,30,100.0,AAAAAAAAAAAAAAAAAAAAAAAAAAAAAA,67


In [23]:
# PolyT tail

# How can certain reads NOT start from the beginning?
(tidehunter
 .loc[lambda x: ~x.cons_seq.str.contains("A|G|C")]
 #.loc[lambda x: x.start < 200]
 #.loc[lambda x: x.copy_number > 10]
 .sort_values("region_length", ascending=False)
)

,id,read_len,start,end,conslen,avematch,cons_seq,region_length
229496,5d484a0c-2f58-4346-b2e1-c2bc22c0fd4f,11309,10352,11054,30,90.7,TTTTTTTTTTTTTTTTTTTTTTTTTTTTTT,702
391944,cf8b14de-67d1-4d04-8d11-fbf6c2a25d41,18648,18234,18648,30,95.1,TTTTTTTTTTTTTTTTTTTTTTTTTTTTTT,414
271696,203f1f0e-0749-49d4-a0a5-6fdff97c9e7e,21091,20680,21090,30,100.0,TTTTTTTTTTTTTTTTTTTTTTTTTTTTTT,410
391855,0e9c39c8-627d-49a1-9d1d-f17c2cd2b610,48189,47819,48188,30,100.0,TTTTTTTTTTTTTTTTTTTTTTTTTTTTTT,369
155942,fd5e4a76-2406-4b25-98d1-5dcd9e2ae4f6,17608,17249,17608,30,100.0,TTTTTTTTTTTTTTTTTTTTTTTTTTTTTT,359
...,...,...,...,...,...,...,...,...
85444,e05c6af4-c2de-4f0e-aae0-d22e04119bc3,4480,4413,4480,30,96.7,TTTTTTTTTTTTTTTTTTTTTTTTTTTTTT,67
91567,d591a109-75fe-42e9-bf1e-c299340764a6,647,53,120,30,78.3,TTTTTTTTTTTTTTTTTTTTTTTTTTTTTT,67
221376,774da64b-17d4-4666-9117-b7e4d0e08997,3613,63,130,30,100.0,TTTTTTTTTTTTTTTTTTTTTTTTTTTTTT,67
537789,6281fd1a-d2fd-4165-b0f5-8008a2b32611,2938,2683,2750,30,100.0,TTTTTTTTTTTTTTTTTTTTTTTTTTTTTT,67


In [24]:
# CCCTAA telomere (how can telomere NOT start at the beginning of read?)
(tidehunter
 .loc[lambda x: x.cons_seq.str.contains("CCCTAACCCTAACCCTAA")]
 #.loc[lambda x: x.start < 200]
 #.loc[lambda x: x.copy_number > 10]
 .sort_values("region_length", ascending=False)
)

,id,read_len,start,end,conslen,avematch,cons_seq,region_length
61337,83c29e37-0983-4e27-aa50-0a9e17cc00fe,29562,92,5411,30,99.2,ACCCTAACCCTAACCCTAACCCTAACCCTA,5319
266572,592662e3-5a6a-4225-aee3-437c2e223907,35123,36,3076,30,99.8,AACCCTAACCCTAACCCTAACCCTAACCCT,3040
290569,b41651fb-5286-49ad-a357-8b0796226c0f,15809,36,2363,30,98.4,AACCCTAACCCTAACCCTAACCCTAACCCT,2327
192994,3f9d457c-b5fa-4abe-aca1-aec18a4a81a2,13628,8386,9814,44,79.6,AACCCTTAACCCCTAACCCTAACCCTAACCCTAACCCTAACCCT,1428
266574,592662e3-5a6a-4225-aee3-437c2e223907,35123,3152,4324,30,97.7,ACCCTAACCCTAACCCTAACCCTAACCCTA,1172
...,...,...,...,...,...,...,...,...
407585,0fd855c2-3e17-4653-87da-fe98c8649792,3725,1574,1711,32,88.5,CCCCTAACCCTAACCCCTAACCCTAACCCTAA,137
392167,02c14fac-6464-4f6b-a4d1-dbb22dbffdea,4586,1471,1601,37,82.1,ACCCTGACCCCTAACCCTAACCCTAACCCTAACCCTA,130
379021,3426318d-69cf-4a00-ad56-0caea3a6120c,15197,33,161,30,96.9,CCTAACCCTAACCCTAACCCTAACCCTAAC,128
465129,a6709ee6-aa50-49df-a313-d1c1ee627c13,433,31,153,30,93.5,ACCCTAACCCTAACCCTAACCCTAACCCTA,122


In [25]:
#TTAGGG telomere
(tidehunter
 .loc[lambda x: x.cons_seq.str.contains("TTAGGGTTAGGGTTAGGG")]
 #.loc[lambda x: x.start < 200]
 #.loc[lambda x: x.copy_number > 10]
 .sort_values("region_length", ascending=False)
)

,id,read_len,start,end,conslen,avematch,cons_seq,region_length
455018,ebf97b90-de41-4368-b006-19533c271081,9662,1961,2956,290,86.3,TTAGTGTTGGGGTTAGGGTTAGGGTTAGGGTATTGGGGTTAGGGTT...,995
433289,34afc44e-331b-49f2-be13-f18a9e35d403,13544,11521,12491,325,85.7,GTTAGGGTTAGGGTTAGGGTTAGGGTTAGGGGTTAGGGTTAGGGTT...,970
424635,b6d1484f-832a-4ed8-a7e2-b917fc5ce5ff,7851,5068,5998,36,90.4,AGGGTTAGGGTTAGGGTTAGGGTTGAGGGTTAGGGT,930
520330,17db409c-ce5a-4684-8e8e-bf5963407248,11864,10278,11029,31,85.7,TTAGGGTTAGGGTTAGGGTTAGGGTTAGGGT,751
193296,dd375ee8-7eb5-46cf-9e73-0a584d9bf6f7,9327,598,1176,60,88.2,TAAGGGTTAGGGTTAGGGTTAGGGTTAGGGTTAGGGTTAGGGTTGG...,578
...,...,...,...,...,...,...,...,...
70549,adcc8e12-02bd-4cfc-98de-dfc4833d915f,2310,1958,2055,30,90.3,GGTTTAGGTTAGGGTTAGGGTTAGGGTGAG,97
415583,8aeab76e-74ff-4d43-9100-5bf1618cd01a,1737,1644,1727,33,100.0,GGTTAGGGGTTAGGGTTAGGGTTAGGGGTTAGG,83
4676,1270cd4f-7ad0-4c2d-9f86-a2e89fc43c55,12562,8820,8900,30,100.0,TTAGGGTTAGGGTTAGGGTTAGGGTTAGGG,80
449112,ec615c0b-b483-480c-8c6e-fe1e0947adea,3310,504,581,30,100.0,TAGGGTTAGGGTTAGGGTTAGGGTTAGGGT,77


# Testing the Tidehunter file with --min-period 12
* the default parameter yielded better result

In [26]:
tidehunter = (
    pd.read_csv("/home/nanopore/analyzed-data/PolyA_2022-11-23/tidehunter/tidehunter_min_period_12.out",
    sep="\t",
    header=None,
    names=["id", "rep_number", "copy_number", "read_len", 
           "start", "end", "conslen", "avematch", "full_len", "subpos", "cons_seq"]
    )
   .assign(region_length=lambda x: x.end - x.start) 
   .drop(columns=["rep_number", "copy_number", "full_len", "subpos"])
) 

In [33]:
(tidehunter
 .loc[lambda x: x.cons_seq.str.contains("CCCTAACCCTAACCCTAA")]
 #.loc[lambda x: x.start < 200]
 #.loc[lambda x: x.copy_number > 10]
 .sort_values("region_length", ascending=False)
)

,id,read_len,start,end,conslen,avematch,cons_seq,region_length
142227,58465f60-c427-4928-894e-580a300f39a6,11723,4930,5810,437,82.4,AGTAATAGTGTAATATAAGTAATAATTTGTTATTAGTAATAATGTG...,880
150497,3f9d457c-b5fa-4abe-aca1-aec18a4a81a2,13628,8095,8819,146,93.0,CCCTTAACCCTAACCCTAACCCTTAACCCTAACCCTAACCCTAACC...,724
147687,f4d2e586-550f-4d74-9708-0f5b8e52c8e1,1743,1242,1518,50,82.8,TCCCAAACCCCTAAACCTAACCCTAATCCTAACCCTAACCCTAACC...,276
162694,c8d5a407-ba02-4450-a315-85e64b3993ec,2962,1349,1588,52,81.3,CCCCTAACCCTAACCCTAACACTTAACCCTAAACCCTAACCCTAAC...,239
291364,0df12648-1e02-4f65-8ae2-75450a163969,2001,931,1132,43,89.3,ACCCTAACCCTAACCCTAAACCTAACCCTAACCCTAAACCTAA,201
94077,e80e6a48-5214-4809-9d5e-1e8138af208f,9138,1303,1502,36,89.1,CCCTAACCCTTACCCTAACCCTAACCCTAACCCTAA,199
249160,711e46e7-e80c-4abe-92ca-139c43f82d94,24009,20834,21033,31,86.2,ACCCTCAACCCTAACCCTAACCCTAACCCTA,199
326064,ea4d6dd4-5c1b-419f-b380-9cc3ce8ed19f,29295,28715,28914,36,89.1,CCCTAACCCTTACCCTAACCCTAACCCTAACCCTAA,199
5166,e6a84e84-426a-4745-b41a-979df7f1a32f,1864,1563,1760,31,83.5,TAACCCTAACCCTAACCCTAACCCCAACCCC,197
388158,23b0d716-3228-4b9c-9ed0-e1dcccf8af71,27296,10353,10550,50,87.5,CCTAGCCCTAACCCTAACCCTAACCCTAACTCTAAAACCCTAACCC...,197


# Merge bed file and tidehunter telomere files

In [8]:
import pandas as pd
left_telomeres = (
    pd.read_csv("/home/nanopore/analyzed-data/PolyA_2022-11-23/tidehunter/left_telomeres.csv")
    [["id", "region_length", "read_len", "start", "end"]]
    .assign(telomere_type="CCCTAA")
    .rename(columns={"start": "tidehunter_start",
             "end": "tidehunter_end"})
)




left_telomeres

,id,region_length,read_len,tidehunter_start,tidehunter_end,telomere_type
0,83c29e37-0983-4e27-aa50-0a9e17cc00fe,5319,29562,92,5411,CCCTAA
1,592662e3-5a6a-4225-aee3-437c2e223907,3040,35123,36,3076,CCCTAA
2,b41651fb-5286-49ad-a357-8b0796226c0f,2327,15809,36,2363,CCCTAA
3,3f9d457c-b5fa-4abe-aca1-aec18a4a81a2,1428,13628,8386,9814,CCCTAA
4,592662e3-5a6a-4225-aee3-437c2e223907,1172,35123,3152,4324,CCCTAA
...,...,...,...,...,...,...
67,0fd855c2-3e17-4653-87da-fe98c8649792,137,3725,1574,1711,CCCTAA
68,02c14fac-6464-4f6b-a4d1-dbb22dbffdea,130,4586,1471,1601,CCCTAA
69,3426318d-69cf-4a00-ad56-0caea3a6120c,128,15197,33,161,CCCTAA
70,a6709ee6-aa50-49df-a313-d1c1ee627c13,122,433,31,153,CCCTAA


In [61]:
bed = (
    pd.read_csv("/home/nanopore/analyzed-data/PolyA_2022-11-23/telomeres/PolyA_2022-11-23_telomeres_raw.bed",
                  sep="\t",
                  header=None,
                  names=["chr", "align_start", "align_end", "id", "x", "strand"])
    .drop(columns=["x"])
)

In [62]:
left_bed = bed.merge(left_telomeres, on="id")
right_bed = bed.merge(right_telomeres, on="id").sort_values("region_length", ascending=False)
left_and_right = pd.concat([left_bed, right_bed]).drop_duplicates((["id", "align_start", "align_end"]))

In [63]:
left_and_right

,chr,align_start,align_end,id,strand,region_length,telomere_type
12,chr5,2,23595,83c29e37-0983-4e27-aa50-0a9e17cc00fe,+,5319,CCCTAA
30,chr10,0,32142,592662e3-5a6a-4225-aee3-437c2e223907,+,3040,CCCTAA
42,chr17,0,16414,b41651fb-5286-49ad-a357-8b0796226c0f,+,2327,CCCTAA
4,chr3,200900163,200913759,3f9d457c-b5fa-4abe-aca1-aec18a4a81a2,-,1428,CCCTAA
18,chr5,182036420,182045435,1be97bf6-7983-4cdf-b1cd-973c598e0045,-,1107,CCCTAA
...,...,...,...,...,...,...,...
28,chr10,8462715,8470121,ee705241-8580-49c8-9db8-f7848b97deb1,-,119,TTAGGG
41,chr12,133312253,133314091,adcc8e12-02bd-4cfc-98de-dfc4833d915f,+,97,TTAGGG
46,chr17,41788,43487,8aeab76e-74ff-4d43-9100-5bf1618cd01a,-,83,TTAGGG
2,chr2,163617380,163620100,ec615c0b-b483-480c-8c6e-fe1e0947adea,-,77,TTAGGG


### TESTING

In [1]:
import pandas as pd

results = pd.read_csv("/home/nanopore/analyzed-data/TeloR30/results/TeloR30_telomere_results.csv")
results

,chr,align_start,align_end,id,strand,region_length,read_len,telomere_pattern_start,telomere_pattern_end,telomere_type
0,chr21,33090786,33106903,a5fa3702-5d6b-40a3-b9cd-ed09783ed180,+,7919,16199,2524,10443,CCCTAA
1,chr3,1386,10225,990b786a-26bf-4c2f-92a4-bf7c380efa47,+,7254,15136,31,7285,CCCTAA
2,chr22,1,20403,5af69638-064f-444c-b2ab-0a2f9d31e158,+,5841,22993,35,5876,CCCTAA
3,chr13,113553272,113566053,2619716f-2b35-4e6a-b2a2-b195790607a3,-,5590,16202,643,6233,CCCTAA
4,chr7,817,16304,7d9dc776-3588-4ecb-9397-ba2e735bd818,+,4279,18627,33,4312,CCCTAA
...,...,...,...,...,...,...,...,...,...,...
650,chr4,50905890,50906262,23b8892d-b099-41ca-840e-87150c9a19f0,-,70,651,34,104,CCCTAA
651,chr7,154066779,154075458,ba53adc0-8dcc-42f6-b822-87e3c0afd5be,-,70,8679,4274,4344,CCCTAA
652,chr1,194014626,194020249,39215cb2-acc1-4761-8f2f-7209a75bd30f,-,70,6145,5719,5789,CCCTAA
653,chrX,81760672,81761322,22cee596-de7a-44d1-bf21-19e6339dd8fe,-,68,984,219,287,CCCTAA


In [2]:
# telomere df
import numpy as np
import pandas as pd

NUM_FROM_END = 10_000

results_telo30 = "/home/nanopore/analyzed-data/TeloR30/results/TeloR30_telomere_results.csv"
results_tag_telo3 = "/home/nanopore/analyzed-data/TagTelo3_221206/results/TagTelo3_221206_telomere_results.csv"
polya = "/home/nanopore/analyzed-data/PolyA_2022-11-23/results/PolyA_2022-11-23_telomere_results.csv"
chr_lengths = pd.read_csv("chm13v2_chr_lengths.csv")

telomere_df = (
    pd.read_csv(
        results_tag_telo3
    )
    .rename(columns={"region_length": "telomere_length"})
    .merge(chr_lengths, on="chr")
    .rename(columns={
        "total_length": "chr_length",
        }
    )
    .assign(
        arm=lambda x: np.select(
            [x.align_start < NUM_FROM_END, x.align_start > x.chr_length - NUM_FROM_END],
            ["left", "right"],
            default=pd.NA,
        )
    )
    .assign(fraction_telomere=lambda x: x.telomere_length / x.read_len)
    .dropna()
    .reset_index()
    .sort_values("telomere_length", ascending=False)
)

telomere_df

,index,chr,align_start,align_end,id,strand,telomere_length,read_len,telomere_pattern_start,telomere_pattern_end,telomere_type,chr_length,arm,fraction_telomere
0,0,chr22,2067,19299,6697258c-f1d1-42d9-bb8b-76005536e7e9,+,6055,20761,56,6111,CCCTAA,51324926,left,0.291653
2,10,chr12,133322268,133322995,4ac04449-b0eb-4a7c-89ff-261620e60f46,-,699,4337,3595,4294,CCCTAA,133324548,right,0.161171
3,12,chr7,2961,18271,d567597e-a639-4b24-8049-4bb93a5038dd,-,497,20265,14662,15159,CCCTAA,160567428,left,0.024525
4,20,chr6,2549,11712,c8108e1d-cb6d-4beb-bc01-41dcda7d6f28,+,331,14957,3276,3607,CCCTAA,172126628,left,0.022130
5,21,chr6,2554,15890,a9c2154a-6fd0-4881-954a-2f2f6d5076dc,+,321,14480,1509,1830,CCCTAA,172126628,left,0.022169
6,22,chr1,2173,12395,a7b89736-9c0d-4d22-ac1d-cd4fd25b1ff1,+,294,17471,4824,5118,CCCTAA,248387328,left,0.016828
7,31,chr13,113562789,113563516,14ec8087-b67e-4e65-9b25-e7f31ef233ea,-,201,783,32,233,CCCTAA,113566686,right,0.256705
8,33,chr2,2151,2696,79b9e178-4787-45d3-b18c-e4192aea8e07,+,153,4751,3746,3899,CCCTAA,242696752,left,0.032204
1,6,chr11,1855,5296,8085a195-e226-4aa1-9f38-560840e608c2,-,130,3909,3513,3643,CCCTAA,135127769,left,0.033257


In [41]:
import altair as alt
from altair_saver import save
def plot_telomeres(telomere_df: pd.DataFrame) -> None:
    """
    Plots information about found telomeres
    """
    plot = (
        alt.Chart(telomere_df)
        .mark_circle()
        .encode(
            alt.X("chr:N", title="Chromosome"),
            alt.Color("telomere_type:N"),
            alt.Y("telomere_length", title="Length of Telomere"),
            alt.Size("read_len:Q", scale=alt.Scale(range=[50, 500]), title= "Read length"),
            tooltip=[alt.Tooltip("telomere_length:Q", title="Telomere length"),
                     alt.Tooltip("telomere_type:N", title="Telomere type"),
                     alt.Tooltip("read_len:Q", title="Read length"),
                     alt.Tooltip("id:N", title="ID"),
                     alt.Tooltip("telomere_pattern_start:Q", title="Start of telomere in read"),
                     alt.Tooltip("telomere_pattern_end:Q", title="End of telomere in read"),
                     alt.Tooltip("fraction_telomere:Q", title="Fraction of telomere / read"),
                    ]
        )
        .properties(
            height=700,
        )
        .facet(column="arm", title=f"Number of telomeric reads: {telomere_df.shape[0]}")
    )
    return plot

plot = plot_telomeres(telomere_df)
save(plot, "telR30.html")
plot

alt.FacetChart(...)

In [1]:
# function to extract fasta file from the above df of telomeres

import pyfastx
import matplotlib.pyplot as plt
import pandas as pd

def extract_fasta_telomeres(fastq: str, telomere_df: pd.DataFrame) -> None:
    """
    """
    telomere_ids = telomere_df["id"].values
    telomere_fastq = pyfastx.Fastq(fastq, build_index=False)
    
    sequence_name = [read[0] for read in telomere_fastq if read[0] in telomere_ids]
    sequence_fasta = [read[1] for read in telomere_fastq if read[0] in telomere_ids]
    # PHRED score
    qualities = [list(read[2]) for read in telomere_fastq if read[0] in telomere_ids]
    qualities = [[ord(x) - 33 for x in y] for y in qualities]
    
    PHRED_df =  (
        pd.DataFrame()
        .assign(
            id=sequence_name,
            fasta=sequence_fasta,
            PHRED=qualities,
        )
    )
    return PHRED_df.merge(telomere_df, on="id")


def plot_telomere_score(phred_df: pd.DataFrame) -> plt.Figure:
    """
    Plots the telomere score
    """
    fig, ax = plt.subplots()
    fig.set_figwidth(15)
    ax.plot(phred_df.PHRED)
    ax.axvspan(
        phred_df.telomere_pattern_start,
        phred_df.telomere_pattern_end, 
        facecolor='lightgreen', 
        alpha=0.4,
    )
    ax.set_ylabel('PHRED score')
    ax.set_xlabel('Position')
    # To put the sequence on the x-axis
    #plt.xticks(list(range(len(phred_df.PHRED))), phred_df.fasta, fontsize=5)
    
    plt.title(f"""
    Read: {phred_df.id}
    Motif: {phred_df.telomere_type}
    Telomere Length: {phred_df.telomere_length}
    """
    )
    return fig, phred_df.fasta
    
    

#fasta_telo_r_30 = extract_fasta_telomeres("/home/nanopore/analyzed-data/TeloR30/telomeres/TeloR30_telomeres_porechop.fastq.gz",
#                        telomere_df) 

fasta_tag_telo_3 = extract_fasta_telomeres("/home/nanopore/analyzed-data/TagTelo3_221206/telomeres/TagTelo3_221206_telomeres_porechop.fastq.gz",
                        telomere_df) 

plot, sequence = plot_telomere_score(fasta_tag_telo_3.iloc[0])
#print(sequence)

#plot.savefig("telomere-3'-example.jpg")

# ONTs way to calculate PHRED:
# https://labs.epi2me.io/notebooks/Introduction_to_fastq_file.html


# open the file and iterate through its records
#with FastxFile("all_records.fastq") as fq:
#    for rec in fq:
#        # ONT calculation for "mean Q score"
#        quals = np.fromiter(
#            (ord(x) - 33 for x in rec.quality),
#            dtype=int, count=len(rec.quality))
#        mean_p = np.mean(np.power(10, quals/-10))
#        mean_qualities.append(-10*np.log10(mean_p))
#        # all qualities
#        qualities.extend(quals)
#        lengths.append(len(quals))

NameError: name 'telomere_df' is not defined

In [8]:
from pathlib import Path

def fast5_pass_files(path):
    in_folder = Path(path)
    fastq_folder = list(in_folder.rglob("*fastq_pass"))
    if fastq_folder:
        return fastq_folder[0]
    else:
        return list(in_folder.rglob("*pass"))[0]
    
path = "/home/nanopore/sequencing-data/PolyA_2022-11-23/"

fast5_pass_files(path)

PosixPath('/home/nanopore/sequencing-data/PolyA_2022-11-23/dna_r10.4.1_e8.2_260bps_sup.cfg/fastq/pass')

# Look at the bed file with all reads aligned and extract those who aligns to the ends of the chromosomes

In [5]:
import pandas as pd
import numpy as np

NUM_FROM_END = 10_000
# telomere df
chr_lengths = pd.read_csv("chm13v2_chr_lengths.csv")
all_reads = (
    pd.read_csv(
        "/home/nanopore/analyzed-data/TagTelo3_221206/all_reads_aligned/TagTelo3_221206_all_reads.bed",
        sep="\t",
        header=None,
        names=["chr", "start", "end", "id", "number", "strand"]
    )
    .assign(read_length=lambda x: x.end - x.start)
    .merge(chr_lengths, on="chr")
    .assign(
        arm=lambda x: np.select(
            [x.start < NUM_FROM_END, x.start > x.total_length - NUM_FROM_END],
            ["left", "right"],
            default=pd.NA,
        )
    )
    .dropna()
    .reset_index()
    .sort_values("start", ascending=True)
)
 



In [10]:
all_reads

,index,chr,start,end,id,number,strand,read_length,total_length,arm
1,98350,chr2,970,1064,d567597e-a639-4b24-8049-4bb93a5038dd,35,-,94,242696752,left
2,98351,chr2,1169,2397,ecb8e2b5-017f-4664-8ce3-1790ae25249c,60,+,1228,242696752,left
30,762732,chr11,1251,19377,8bdfea17-968e-4d83-bd54-2b7bf5f198f3,31,+,18126,135127769,left
47,920228,chr14,1421,11485,23724348-c135-4083-96df-2e96582f2adf,60,-,10064,101161492,left
31,762733,chr11,1797,19619,c942085e-1fc1-45f1-acba-2ada9fce9e45,42,-,17822,135127769,left
...,...,...,...,...,...,...,...,...,...,...
8,449689,chr5,182039945,182041197,e856bd5e-1a3e-4540-8b5b-877e3b279689,17,-,1252,182045439,right
6,371622,chr4,193565041,193567021,427e4505-bb53-4e93-9765-55c69c15bcbc,60,+,1980,193574945,right
7,371623,chr4,193571626,193571851,6112fa51-c267-400f-ba96-b03a7ab0e1fb,32,+,225,193574945,right
4,288066,chr3,201096381,201101788,8939369b-7d48-4217-b895-a75edf0f98d8,60,-,5407,201105948,right


# Look at the pure telomere reads (from tidehunter)
### Not aligned to the reference
### Telomeres with fraction > 0.7 of the whole read

In [25]:
left = pd.read_csv("/home/nanopore/analyzed-data/TeloR30//tidehunter/left_telomeres.csv")
right = pd.read_csv("/home/nanopore/analyzed-data/TeloR30//tidehunter/right_telomere.csv")
both = pd.concat([left, right])
both

,id,start,end,read_len,cons_seq,region_length
0,a5fa3702-5d6b-40a3-b9cd-ed09783ed180,2524,10443,16199,CTAACCCTAACCCTAACCCTAACCCTAACC,7919
1,990b786a-26bf-4c2f-92a4-bf7c380efa47,31,7285,15136,AACCCTAACCCTAACCCTAACCCTAACCCT,7254
2,5af69638-064f-444c-b2ab-0a2f9d31e158,35,5876,22993,ACCCTAACCCTAACCCTAACCCTAACCCTA,5841
3,2619716f-2b35-4e6a-b2a2-b195790607a3,643,6233,16202,TAACCCTAACCCTAACCCTAACCCTAACCC,5590
4,7d9dc776-3588-4ecb-9397-ba2e735bd818,33,4312,18627,AACCCTAACCCTAACCCTAACCCTAACCCT,4279
...,...,...,...,...,...,...
179,e7e449c8-7b71-42ac-9003-247dbb4cbd00,34,110,241,GTTAGTTAGGGTTAGGGTTAGGGTTAGGGTTAGG,76
180,6b7966d5-c3d3-463e-922f-f52e76a5db0e,269,344,348,AGGGTTAGGGTTAGGGTTAGGGTTAGGGTT,75
181,f485e625-6f50-4755-8a31-c72b148795a8,957,1029,1188,GGGTTAGGGTTAGGGTTAGGGTTAGGGTTA,72
182,ba53adc0-8dcc-42f6-b822-87e3c0afd5be,4274,4344,8679,GTTAGGGTTAGGGTTAGGGTTAGGGTTAGG,70


In [24]:
just_telomeres = (
    both
    .assign(fraction_telomere=lambda x: x.region_length / x.read_len)
    .assign(n_telomeres=lambda x: x.shape[0])
    .loc[lambda x: x.fraction_telomere > 0.7]
)

just_telomeres
#just_telomeres.to_excel("just-telomres.xlsx")

,id,start,end,read_len,cons_seq,region_length,fraction_telomere,n_telomeres
26,2fd56dac-cadd-4c26-9913-87a50eecac4e,35,628,669,CTAACCCTAACCCTAACCCTAACCCTAACC,593,0.886398,939
48,be4040d9-4ee2-4145-bec7-7f6493763636,39,512,571,AACCCTAACCCTAACCCTAACCCTAACCCT,473,0.828371,939
50,69b9369b-419c-4d47-b108-77fb8327784e,30,498,565,AACCCTAACCCTAACCCTAACCCTAACCCT,468,0.828319,939
57,7b7b7e26-ac47-4915-9300-6925572d1038,31,472,547,AACCCTAACCCTAACCCTAACCCTAACCCT,441,0.806216,939
64,f9229efe-4591-42f1-abd6-ba18bd682c59,30,452,494,AACCCTAACCCTAACCCTAACCCTAACCCT,422,0.854251,939
...,...,...,...,...,...,...,...,...
29,e08006ba-3c5f-4d79-8d68-228c992641bd,87,348,356,GTTAGGGTTAGGGTTAGGGTTAGGGTTAGG,261,0.733146,939
35,4cab4751-fe33-4bce-b51c-26ad5905771a,85,333,339,GGGTTAGGGTTAGGGTTAGGGTTAGGGTTA,248,0.731563,939
48,ec7789da-b743-489f-98f5-73ccfc034a81,4,225,232,GGGTTAGGGTTAGGGTTAGGGTTAGGGTTA,221,0.952586,939
73,1706ac50-4d33-42df-b891-7ea726403bba,35,218,222,GGGTTAGGGTTAGGGTTAGGGTTAGGGTTA,183,0.824324,939


## Change name on the chromosome on the T2T-reference

In [1]:
with open("../../reference-genomes/t2t-chm13v2/T2T-CHM13v2.0_genomic.fna", "r") as f:
    reference = f.readlines()
    


In [2]:
with open("../../reference-genomes/t2t-chm13v2/T2T-CHM13v2.0_new_chr_name.fna", "a+") as f:
    index = 0
    chromosomes = [f"chr{x}" for x in list(range(1, 22 + 1)) + ["X", "Y"]]
    for line in reference:
        if line.startswith(">"):
            print(line)
            line = f">{chromosomes[index]}\n"
            f.write(line)
            index += 1
        else:
            f.write(line)
         

>NC_060925.1 Homo sapiens isolate CHM13 chromosome 1, alternate assembly T2T-CHM13v2.0

>NC_060926.1 Homo sapiens isolate CHM13 chromosome 2, alternate assembly T2T-CHM13v2.0

>NC_060927.1 Homo sapiens isolate CHM13 chromosome 3, alternate assembly T2T-CHM13v2.0

>NC_060928.1 Homo sapiens isolate CHM13 chromosome 4, alternate assembly T2T-CHM13v2.0

>NC_060929.1 Homo sapiens isolate CHM13 chromosome 5, alternate assembly T2T-CHM13v2.0

>NC_060930.1 Homo sapiens isolate CHM13 chromosome 6, alternate assembly T2T-CHM13v2.0

>NC_060931.1 Homo sapiens isolate CHM13 chromosome 7, alternate assembly T2T-CHM13v2.0

>NC_060932.1 Homo sapiens isolate CHM13 chromosome 8, alternate assembly T2T-CHM13v2.0

>NC_060933.1 Homo sapiens isolate CHM13 chromosome 9, alternate assembly T2T-CHM13v2.0

>NC_060934.1 Homo sapiens isolate CHM13 chromosome 10, alternate assembly T2T-CHM13v2.0

>NC_060935.1 Homo sapiens isolate CHM13 chromosome 11, alternate assembly T2T-CHM13v2.0

>NC_060936.1 Homo sapiens isol

## Copying files from analysis dir except bam, fastq, bai files

In [3]:
import pathlib
import shutil


shutil.copytree(src, dst, shutil.ignore_patterns([".bai", ".bam", ".fastq", ".fastq.gz"]))
